## Collect temperature data from NDAWN & run interpolations.
https://ndawn.ndsu.nodak.edu/

In [25]:
import pandas as pd
import requests
from datetime import date
from io import StringIO

import arcpy
arcpy.env.workspace = r'C:\Users\mmMary\Documents\GIS_Classes\GIS_5572\Labs\Lab4\lab4_map_ndawn\lab4_map_ndawn.gdb'

In [26]:
# sample code for rolling dates
import pandas as pd
from datetime import datetime, timedelta, date

# https://www.codegrepper.com/code-examples/python/create+list+of+last+30+days+in+python

In [27]:
# test date range

DateStart = date.today().isoformat()  
DateRolling = (date.today() - timedelta(days=30)).isoformat()
print(DateRolling)

2021-03-19


In [28]:
# Construct API call for the request
base = 'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=142&station=138&station=9&station=10&station=118&station=56&station=11&station=12&station=58&station=13&station=84&station=55&station=7&station=87&station=14&station=15&station=96&station=16&station=137&station=124&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=19&station=129&station=20&station=101&station=81&station=21&station=97&station=22&station=75&station=2&station=139&station=23&station=62&station=86&station=24&station=89&station=126&station=93&station=90&station=25&station=83&station=107&station=77&station=26&station=70&station=127&station=27&station=132&station=28&station=29&station=30&station=31&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=105&station=82&station=34&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=71&station=37&station=38&station=39&station=130&station=73&station=40&station=41&station=54&station=69&station=113&station=128&station=42&station=43&station=103&station=116&station=88&station=114&station=3&station=64&station=115&station=67&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=74&station=60&station=125&station=8&station=47&station=122&station=108&station=5&station=48&station=68&station=49&station=50&station=91&station=117&station=63&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=57&station=110&variable=ddavt&year=2021&ttype=daily&'
dates = 'quick_pick=&begin_date='
        
# Get date range
DateStart = date.today().isoformat()  
DateEnd = (date.today() - timedelta(days=30)).isoformat()

URL = str(base + dates + DateEnd +'&end_date=' + DateStart)
        
             
# Request page
resp = requests.get(URL) # Make API call
url_content = resp.content  # Get content
csv_file = open('30Days_temp.csv', 'wb')# Save file to disc
csv_file.write(url_content) # Get file size

# # If status code not 200, raise exception
# if page.status_code != 200:
#     raise Exception('URL request status not 200. Status code = ' + page.status_code)

#     print('Request successful')

202761

In [29]:
# Read the table & disregard the top 3 rows of metadata
test = pd.read_csv('30Days_temp.csv', skiprows = 3)
test

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Temp,Avg Temp Flag
0,NaN,deg,deg,ft,NaN,NaN,NaN,Degrees F,NaN
1,Ada,47.3211,-96.5139,910,2021.0,3.0,19.0,40.395,NaN
2,Ada,47.3211,-96.5139,910,2021.0,3.0,20.0,50.546,NaN
3,Ada,47.3211,-96.5139,910,2021.0,3.0,21.0,42.461,NaN
4,Ada,47.3211,-96.5139,910,2021.0,3.0,22.0,35.165,NaN
...,...,...,...,...,...,...,...,...,...
3926,Zeeland,46.013378,-99.687587,2070,2021.0,4.0,13.0,24.417,NaN
3927,Zeeland,46.013378,-99.687587,2070,2021.0,4.0,14.0,31.173,NaN
3928,Zeeland,46.013378,-99.687587,2070,2021.0,4.0,15.0,37.556,NaN
3929,Zeeland,46.013378,-99.687587,2070,2021.0,4.0,16.0,36.441,NaN


## The table must be manually edited to remove the top 3 rows of metadata before the monthly mean can be calculated. 
### There has to be a good way to do this here but that's undiscovered at this point. 

In [43]:
# Run statistics summary to get one monthly temp average value for
# each station, preserving lat, long, & elevation values also.

rcpy.analysis.Statistics("last30Days_temp", r"C:\Users\mmMary\Documents\GIS_Classes\GIS_5572\Labs\Lab4\lab4_map_ndawn\lab4_map_ndawn.gdb\Temp_ave", "Avg_Temp_F MEAN", None)

<Result 'C:\\Users\\mmMary\\Documents\\GIS_Classes\\GIS_5572\\Labs\\Lab4\\lab4_map_ndawn\\lab4_map_ndawn.gdb\\Temp_ave'>

### Display station points & label with the mean temperature

In [ ]:
# Map station points with display xy data  
# Convert coordinates into points

arcpy.management.XYTableToPoint("last30Days_temp.csv", "station_pts", "Longitude", "Latitude deg", None, "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision")

In [ ]:
# Add stations layer 
stations = arcpy.mp.lyr("station_pts.shp")

In [ ]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
for m in aprx.listMaps("Map"):
    for lyr in m.listLayers("station_pts"):
        lblClass = lyr.listLabelClasses()[0]
        print(lblClass.name)
        lblClass.expression = "$feature.MEAN_Ave_T"
        lyr.showLabels = True


## Interpolation Methods 
### with esri default parameters.

In [ ]:
# Inverse Distance Weighted method
out_raster = arcpy.sa.Idw("station_Temp_ave_XYTableToPoint", "Latitude_deg", 0.014477764, 2, "VARIABLE 12", None); out_raster.save(r"C:\Users\mmMary\Documents\GIS_Classes\GIS_5572\Labs\Lab4\lab4_map_ndawn\lab4_map_ndawn.gdb\Idw_station_1")

In [ ]:
# Kriging
out_surface_raster = arcpy.sa.Kriging("station_pts", "MEAN_Avg_Temp__F", "Spherical 0.014478 # # #", 0.014477764, "VARIABLE 12", None); out_surface_raster.save(r"C:\Users\mmMary\Documents\GIS_Classes\GIS_5572\Labs\Lab4\lab4_map_ndawn\lab4_map_ndawn.gdb\Kriging")

In [ ]:
# Radial Basis Function
arcpy.ga.RadialBasisFunctions("station_pts", "MEAN_Avg_Temp__F", "RBF_temp", r"C:\Users\mmMary\Documents\GIS_Classes\GIS_5572\Labs\Lab4\lab4_map_ndawn\lab4_map_ndawn.gdb\RBF_elev_spline", 0.014477764, "NBRTYPE=StandardCircular RADIUS=2.89314365250512 ANGLE=0 NBR_MAX=15 NBR_MIN=5 SECTOR_TYPE=ONE_SECTOR", "COMPLETELY_REGULARIZED_SPLINE", None)

In [ ]:
# Map station points with display xy data  
# Convert coordinates into points
arcpy.management.XYTableToPoint("station_stats", r"C:\Users\mmMary\Documents\GIS_Classes\GIS_5572\Labs\Lab4\lab4_map_ndawn\lab4_map_ndawn.gdb\station_pts", "Longitude", "Latitude_deg", "Elevation_ft", "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],VERTCS['WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PARAMETER['Vertical_Shift',0.0],PARAMETER['Direction',1.0],UNIT['Meter',1.0]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision")

https://www.earthdatascience.org/courses/intro-to-earth-data-science/scientific-data-structures-python/pandas-dataframes/import-csv-files-pandas-dataframes/

https://stackoverflow.com/questions/18039057/python-pandas-error-tokenizing-data

https://github.umn.edu/BISH0227/NDAWN_Weather_Scraper